In [ ]:
import sys
from helper import *
from datetime import datetime, timedelta
from indicators import SslChannel, ExponentialMovingAverage
from strategies import SslChannelEmaStrategy
import mplfinance as mpf
from api.api import *

In [ ]:
ssl_channel_ema_strategy = SslChannelEmaStrategy()

In [ ]:
candlestick_5m, candlestick_1h = ssl_channel_ema_strategy.calculate_current_trend()

In [ ]:
ssl_channel = SslChannel()
ema = ExponentialMovingAverage()

In [ ]:
candlestick_df_ssl = ssl_channel.apply_indicator(candlestick_5m)
candlestick_df_ssl_ema = ema.apply_indicator(candlestick_df_ssl)

In [ ]:
candlestick_df_ssl_ema.tail()

In [ ]:
# Plots
apds = [
    mpf.make_addplot(candlestick_df_ssl_ema[['ssl_channel_ssl_down']], color='violet'),
    mpf.make_addplot(candlestick_df_ssl_ema[['ssl_channel_ssl_up']], color='blue'),
    mpf.make_addplot(candlestick_df_ssl_ema[['ema_10']], color='orange'),
    mpf.make_addplot(candlestick_df_ssl_ema[['ssl_channel_trend']])
]


mpf.plot(candlestick_df_ssl_ema, type='candle', style='charles', addplot=apds)

In [ ]:
from connectors import BitstampConnector,DummyConnector

In [ ]:
bitstamp_connector = BitstampConnector()
dummy_connector =DummyConnector()

In [ ]:
bitstamp_connector.get_balance()

In [ ]:
dummy_connector.get_balance()

In [ ]:
dummy_connector.buy_eth(0.03, 1000)

In [ ]:
dummy_connector.get_balance()

In [ ]:
dummy_connector.sell_eth(0.05, 1000)
dummy_connector.get_balance()

In [ ]:
bitstamp_connector.buy_eth(0.05, 1000)

In [ ]:
bitstamp_connector.sell_eth(0.04, 10000)

In [ ]:
# Initial account-balance
from models import InfluxConnector, AccountBalance
from datetime import datetime

influx_connector = InfluxConnector()
client = influx_connector.client

current_account_balance = AccountBalance(timestamp_utc=datetime.utcnow(), pair="ETH-EUR", exchange="Bitstamp", eth_available=0, eur_available=500, balance_total=500)
influx_connector.write_point(current_account_balance.to_influx(connector="simulator"))


In [ ]:
from connectors import BitstampConnector,DummyConnector

dummy_connector = DummyConnector()
current_balance = dummy_connector.get_balance()

print("current_balance.eur_available",current_balance.eur_available)


In [ ]:
dummy_connector.buy_eth(0.05,1000)

In [ ]:
dummy_connector.sell_eth(0.03,1050)

In [ ]:
dummy_connector.sell_eth(0.02,1100)

In [ ]:
from connectors import DummyConnector
from strategies import SimpleStrategy

dummy_connector = DummyConnector()
simple_strategy = SimpleStrategy()


In [ ]:
from api.api import get_eth_eur_values
from helper import create_candlesticks
import mplfinance as mpf
live_trades= get_eth_eur_values(from_dt_str="now()-4d")
candlestick_5m = create_candlesticks(live_trades)

In [ ]:
candlestick_5m.tail()

In [ ]:
candlestick_5m= candlestick_5m.set_index('timestamp_utc')

In [ ]:
mpf.plot(candlestick_5m, type='candle', style='charles')

In [ ]:
import talib


In [ ]:
candle_names = talib.get_function_groups()['Pattern Recognition']

In [ ]:
candlestick_5m['engulfing'] = talib.CDLENGULFING(candlestick_5m['open'],candlestick_5m['high'], candlestick_5m['low'], candlestick_5m['close'])

In [ ]:
candlestick_5m.loc[candlestick_5m.engulfing>0]

In [ ]:
#candlestick_5m.tail(50)

In [ ]:
candlestick_5m['ema_10'] = talib.EMA( candlestick_5m['close'],10)
candlestick_5m['ema_20'] = talib.EMA( candlestick_5m['close'],20)

In [ ]:
candlestick_5m.tail(50)

In [ ]:
# Plots
apds = [
    mpf.make_addplot(candlestick_5m[['ema_10']], color='violet'),
    mpf.make_addplot(candlestick_5m[['ema_20']], color='orange'),
]
mpf.plot(candlestick_5m, type='candle', style='charles', addplot=apds)

In [ ]:
from connectors import BitstampConnector,DummyConnector

dummy_connector = DummyConnector()
current_balance = dummy_connector.get_last_transaction(type='sell')
print("current_balance.amount",current_balance.amount)

bitstamp_connector = BitstampConnector()
last_transaction= dummy_connector.get_last_transaction()
if last_transaction.id is None:
    print("empty transactions")
else:
    print("last_transaction.id",last_transaction.id)

In [21]:
from connectors import DummyConnector
from strategies import SimpleStrategy

dummy_connector = DummyConnector()
simple_strategy = SimpleStrategy()

influx_client <influxdb.client.InfluxDBClient object at 0x7f1a3c388650>
type(result_points[idx]) <class 'dict'>
influx_connector InfluxConnector(client=<influxdb.client.InfluxDBClient object at 0x7f1a3c305f10>)
account_balance.to_influx() {'measurement': 'simulator_account_balance', 'tags': {'pair': 'ETH-EUR', 'exchange': 'Bitstamp'}, 'time': datetime.datetime(2021, 8, 31, 7, 58, 36, 925856), 'fields': {'eur_available': 1000.0, 'eth_available': 1.5, 'balance_total': 5253.61}}


In [22]:
data = simple_strategy.apply(dummy_connector)

influx_client <influxdb.client.InfluxDBClient object at 0x7f1a3c3056d0>
type(result_points[idx]) <class 'dict'>
live_trades 1254
influx_client <influxdb.client.InfluxDBClient object at 0x7f1a3c48cb10>
type(result_points[idx]) <class 'dict'>
current eth-eur-value: 2832.89
query_str SELECT * FROM simulator_transactions order by time desc limit 1
out-trade


In [1]:
data.tail(31)

NameError: name 'data' is not defined

In [ ]:
data.loc[data.index >"2021-08-27 20:50"]

In [ ]:
data.tail()

In [ ]:
data[:-1]

In [ ]:
last_records= data[-2:]

In [ ]:
len(last_records.loc[last_records['engulfing']])>0

In [5]:
from connectors import DummyConnector, BitstampConnector
dummy_connector = DummyConnector()
bitstamp_connector = BitstampConnector()
current_dummy_balance = dummy_connector.get_balance()
current_bitstamp_balance = bitstamp_connector.get_balance()

influx_client <influxdb.client.InfluxDBClient object at 0x7fd218c2e310>
type(result_points[idx]) <class 'dict'>
influx_connector InfluxConnector(client=<influxdb.client.InfluxDBClient object at 0x7fd218ae6610>)
account_balance.to_influx() {'measurement': 'simulator_account_balance', 'tags': {'pair': 'ETH-EUR', 'exchange': 'Bitstamp'}, 'time': datetime.datetime(2021, 8, 29, 20, 32, 30, 288162), 'fields': {'eur_available': 503.5, 'eth_available': 0.0, 'balance_total': 503.5}}
influx_client <influxdb.client.InfluxDBClient object at 0x7fd2189b9890>
type(result_points[idx]) <class 'dict'>
influx_connector InfluxConnector(client=<influxdb.client.InfluxDBClient object at 0x7fd219f93a90>)
account_balance.to_influx() {'measurement': 'bitstamp_account_balance', 'tags': {'pair': 'ETH-EUR', 'exchange': 'Bitstamp'}, 'time': datetime.datetime(2021, 8, 29, 20, 32, 30, 560265), 'fields': {'eur_available': 0.85, 'eth_available': 2.14, 'balance_total': 5882.26}}


In [6]:
dummy_connector.tradeable_eth()

self.eth_reserve 1.5


-1.5

In [7]:
bitstamp_connector.tradeable_eth()

self.eth_reserve 1.5


0.6446775800000002

In [7]:
current_bitstamp_balance.eur_available

0.85

In [12]:
from api.api import get_current_eth_eur_value
current_eth_eur_value = getattr(get_current_eth_eur_value(),'price')

influx_client <influxdb.client.InfluxDBClient object at 0x7fd21893e410>
type(result_points[idx]) <class 'dict'>


In [13]:
current_eth_eur_value

2744.46

In [23]:
last_records= data[-2:]

NameError: name 'data' is not defined

In [4]:
last_records = data[-2:-1]
last_relevant_record = data[-2:-1]
predecessor_record = data[-3:-2]

In [17]:
length_predecessor_candle = abs(predecessor_record['open'].values[0]-predecessor_record['close'].values[0])

In [20]:
last_relevant_record = data[-2:-1].values
last_relevant_record

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [16]:
0.2*length_predecessor_candle

timestamp_utc
2021-08-30 23:55:00+00:00    0.414
Name: close, dtype: float64

In [18]:
if ( last_relevant_record['close'].values[0]< predecessor_record['open'].values[0] + 0.2*length_predecessor_candle):
    print('true')
else:
    print("false")

true


In [9]:
print("last buying price",5, "current_eth_eur_value",6)

last buying price 5 current_eth_eur_value 6


In [1]:
# Initial account-balance
from models import InfluxConnector, AccountBalance
from datetime import datetime

influx_connector = InfluxConnector()
client = influx_connector.client

current_account_balance = AccountBalance(timestamp_utc=datetime.utcnow(), pair="ETH-EUR", exchange="Bitstamp", eth_available=1.5, eur_available=1000, balance_total=5247.65)
influx_connector.write_point(current_account_balance.to_influx(connector="simulator"))

In [26]:
import pandas as pd
import numpy as np
df = pd.DataFrame(np.random.randn(10,3),columns=['A','B','C'])
df = df.set_index('A')

In [24]:
df = df.reset_index()

In [17]:
max(df.loc[df['B'] > 0.390563].idxmax())

-1.4169540763379864

In [8]:
import os
from binance.client import Client

# init
api_key = os.environ.get('BINANCE_API_KEY')
api_secret = os.environ.get('BINANCE_API_SECRET')


client = Client(api_key, api_secret)

# testnet
# client.API_URL = 'https://testnet.binance.vision/api'

# actual
# client.API_URL = 'https://api.binance.com/api'

print("client.API_URL", client.API_URL)
# get balances for all assets & some account information
# print(client.get_account())

res = client.get_asset_balance(asset='EUR')
print(res)


client.API_URL https://api.binance.com/api
{'asset': 'EUR', 'free': '1099.60187480', 'locked': '0.00000000'}


In [10]:
float(res['free'])

1099.6018748

In [5]:
print(client.get_account())

{'makerCommission': 0, 'takerCommission': 0, 'buyerCommission': 0, 'sellerCommission': 0, 'canTrade': True, 'canWithdraw': False, 'canDeposit': False, 'updateTime': 1631133793811, 'accountType': 'SPOT', 'balances': [{'asset': 'BNB', 'free': '1000.00000000', 'locked': '0.00000000'}, {'asset': 'BTC', 'free': '1.00000000', 'locked': '0.00000000'}, {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'}, {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'}, {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'}, {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'}, {'asset': 'USDT', 'free': '10000.00000000', 'locked': '0.00000000'}, {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}], 'permissions': ['SPOT']}


In [10]:
print(client.get_asset_balance(asset='ETH'))

{'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'}


In [1]:
api_key = os.environ.get('BINANCE_API_KEY')
api_secret = os.environ.get('BINANCE_API_SECRET')

client = Client(api_key, api_secret)
res = client.get_asset_balance(asset='ETH')
print(res)

NameError: name 'Client' is not defined

In [3]:
from  datetime import datetime
number = 1631564923
print(datetime.fromtimestamp(int(number)))

2021-09-13 22:28:43


In [9]:
import operator
from dataclasses import dataclass

@dataclass
class test:
    a:int
    b: int

obj_a = test(a=1,b=2)
obj_b = test(a=2,b=4)
obj_c = test(a=3,b=5)

lst = [
obj_a, obj_b, obj_c
]
print(min(lst, key=operator.attrgetter('a')))

test(a=1, b=2)


In [16]:
from connectors import BinanceConnector

binance_connector = BinanceConnector()
binance_connector.get_balance()

client <binance.client.Client object at 0x7f70d086bbd0>
current_etheur_value 2904.14
account_balance.to_influx() {'measurement': 'binance_account_balance', 'tags': {'pair': 'ETH-EUR', 'exchange': 'Binance'}, 'time': datetime.datetime(2021, 9, 18, 20, 58, 54, 162251), 'fields': {'eur_available': 811.0, 'eth_available': 0.12, 'balance_total': 1162.05}}


AccountBalance(timestamp_utc=datetime.datetime(2021, 9, 18, 20, 58, 54, 152918), pair='ETH-EUR', exchange='Binance', eth_available=0.12087649, eur_available=811.0038748, balance_total=1162.0461244685998)

In [4]:
import os
from binance.client import Client
from binance.enums import *

# init
api_key = os.environ.get('BINANCE_API_KEY')
api_secret = os.environ.get('BINANCE_API_SECRET')


client = Client(api_key, api_secret)

# testnet
# client.API_URL = 'https://testnet.binance.vision/api'

# actual
# client.API_URL = 'https://api.binance.com/api'

print("client.API_URL", client.API_URL)
# get balances for all assets & some account information
# print(client.get_account())

res = client.get_asset_balance(asset='EUR')
print(res)

# order = client.order_limit_buy(
#     symbol='ETHEUR',
#     timeInForce='IOC',
#     quantity=0.01,
#     price=2500)

# order = client.create_test_order(
#     symbol='ETHEUR',
#     side='BUY',
#     type=ORDER_TYPE_LIMIT,
#     timeInForce=TIME_IN_FORCE_GTC,
#     quantity=0.01,
#     price=2500)


client.API_URL https://api.binance.com/api
{'asset': 'EUR', 'free': '811.00387480', 'locked': '0.00000000'}


In [9]:
order

{'symbol': 'ETHEUR',
 'orderId': 961851527,
 'orderListId': -1,
 'clientOrderId': 'sQlu6VnSwgMBLzSfKcr0YG',
 'transactTime': 1631912486215,
 'price': '2500.00000000',
 'origQty': '0.01000000',
 'executedQty': '0.00000000',
 'cummulativeQuoteQty': '0.00000000',
 'status': 'EXPIRED',
 'timeInForce': 'IOC',
 'type': 'LIMIT',
 'side': 'BUY',
 'fills': []}

In [5]:
orders = client.get_all_orders(symbol='ETHEUR')

In [8]:
current_order = [x for x in orders if x['orderId'] == 961851527][0]

In [14]:
from models import Transaction
from datetime import datetime
import pytz
transaction = Transaction(timestamp_utc=datetime.fromtimestamp(current_order['time']/ 1000.0).astimezone(pytz.utc)
                                                , exchange="Binance", pair="ETH-EUR", amount=float(current_order['executedQty'])
                                                , price=float(current_order['price'])
                                                ,id =str(current_order['orderId'])
                                                ,type="buy" )

In [13]:
current_order['symbol']

'ETHEUR'

In [9]:
current_order

{'symbol': 'ETHEUR',
 'orderId': 961851527,
 'orderListId': -1,
 'clientOrderId': 'sQlu6VnSwgMBLzSfKcr0YG',
 'price': '2500.00000000',
 'origQty': '0.01000000',
 'executedQty': '0.00000000',
 'cummulativeQuoteQty': '0.00000000',
 'status': 'EXPIRED',
 'timeInForce': 'IOC',
 'type': 'LIMIT',
 'side': 'BUY',
 'stopPrice': '0.00000000',
 'icebergQty': '0.00000000',
 'time': 1631912486215,
 'updateTime': 1631912486215,
 'isWorking': True,
 'origQuoteOrderQty': '0.00000000'}

In [11]:
orders

[{'symbol': 'ETHEUR',
  'orderId': 954234039,
  'orderListId': -1,
  'clientOrderId': 'web_7c303d391aec4a258834eaab790aa0c6',
  'price': '3009.10000000',
  'origQty': '0.34000000',
  'executedQty': '0.34000000',
  'cummulativeQuoteQty': '1023.32520000',
  'status': 'FILLED',
  'timeInForce': 'GTC',
  'type': 'LIMIT',
  'side': 'SELL',
  'stopPrice': '0.00000000',
  'icebergQty': '0.00000000',
  'time': 1631730503177,
  'updateTime': 1631730503177,
  'isWorking': True,
  'origQuoteOrderQty': '0.00000000'},
 {'symbol': 'ETHEUR',
  'orderId': 958064387,
  'orderListId': -1,
  'clientOrderId': 'QNdzIw36rSvvipjyZ6vRuQ',
  'price': '2900.00000000',
  'origQty': '0.01000000',
  'executedQty': '0.00000000',
  'cummulativeQuoteQty': '0.00000000',
  'status': 'CANCELED',
  'timeInForce': 'GTC',
  'type': 'LIMIT',
  'side': 'BUY',
  'stopPrice': '0.00000000',
  'icebergQty': '0.00000000',
  'time': 1631823395511,
  'updateTime': 1631823556657,
  'isWorking': True,
  'origQuoteOrderQty': '0.000000

In [17]:
orders[0]['orderId']

[{'symbol': 'ETHEUR',
  'orderId': 958064387,
  'orderListId': -1,
  'clientOrderId': 'QNdzIw36rSvvipjyZ6vRuQ',
  'price': '2900.00000000',
  'origQty': '0.01000000',
  'executedQty': '0.00000000',
  'cummulativeQuoteQty': '0.00000000',
  'status': 'NEW',
  'timeInForce': 'GTC',
  'type': 'LIMIT',
  'side': 'BUY',
  'stopPrice': '0.00000000',
  'icebergQty': '0.00000000',
  'time': 1631823395511,
  'updateTime': 1631823395511,
  'isWorking': True,
  'origQuoteOrderQty': '0.00000000'}]

In [18]:
result = client.cancel_order(
    symbol='ETHEUR',
    orderId=orders[0]['orderId'])

In [22]:
order

{'symbol': 'ETHEUR',
 'orderId': 958064387,
 'orderListId': -1,
 'clientOrderId': 'QNdzIw36rSvvipjyZ6vRuQ',
 'transactTime': 1631823395511,
 'price': '2900.00000000',
 'origQty': '0.01000000',
 'executedQty': '0.00000000',
 'cummulativeQuoteQty': '0.00000000',
 'status': 'NEW',
 'timeInForce': 'GTC',
 'type': 'LIMIT',
 'side': 'BUY',
 'fills': []}

In [19]:
from datetime import datetime
import pytz
datetime.fromtimestamp(order['transactTime']/ 1000.0).astimezone(pytz.utc)


datetime.datetime(2021, 9, 17, 21, 1, 26, 215000, tzinfo=<UTC>)

In [1]:
from connectors import BinanceConnector
binance_connector = BinanceConnector()
binance_connector.account_balance

binance_connector.buy_eth(0.01, 2886)

client <binance.client.Client object at 0x7f71082089d0>
current_etheur_value 2885.62
account_balance.to_influx() {'measurement': 'binance_account_balance', 'tags': {'pair': 'ETH-EUR', 'exchange': 'Binance'}, 'time': datetime.datetime(2021, 9, 17, 21, 35, 11, 337531), 'fields': {'eur_available': 1099.6, 'eth_available': 0.02, 'balance_total': 1160.13}}
base_price 2885.62
bidding_value 2885.62
client <binance.client.Client object at 0x7f70d3015110>
{'symbol': 'ETHEUR', 'orderId': 961930562, 'orderListId': -1, 'clientOrderId': 'z8DE4sCxQ5A54wNtpJOyFY', 'transactTime': 1631914511719, 'price': '2885.62000000', 'origQty': '0.10000000', 'executedQty': '0.00000000', 'cummulativeQuoteQty': '0.00000000', 'status': 'EXPIRED', 'timeInForce': 'FOK', 'type': 'LIMIT', 'side': 'BUY', 'fills': []}
order {'symbol': 'ETHEUR', 'orderId': 961930562, 'orderListId': -1, 'clientOrderId': 'z8DE4sCxQ5A54wNtpJOyFY', 'transactTime': 1631914511719, 'price': '2885.62000000', 'origQty': '0.10000000', 'executedQty': 

KeyError: 'transactTime'

In [3]:
orders

NameError: name 'orders' is not defined

In [2]:
from connectors import BinanceConnector
binance_connector = BinanceConnector()
binance_connector.account_balance

binance_connector.sell_eth(0.1, 2900)

client <binance.client.Client object at 0x7f267c32aed0>
current_etheur_value 2901.6
account_balance.to_influx() {'measurement': 'binance_account_balance', 'tags': {'pair': 'ETH-EUR', 'exchange': 'Binance'}, 'time': datetime.datetime(2021, 9, 18, 21, 8, 39, 498862), 'fields': {'eur_available': 811.0, 'eth_available': 0.12, 'balance_total': 1161.74}}
base_price 2901.6
bidding_value 2901.6
client <binance.client.Client object at 0x7f267c198610>
{'symbol': 'ETHEUR', 'orderId': 965165814, 'orderListId': -1, 'clientOrderId': 'JrQOEd4XdpNlznfEAEyDbQ', 'transactTime': 1631999318981, 'price': '2901.60000000', 'origQty': '0.10000000', 'executedQty': '0.10000000', 'cummulativeQuoteQty': '290.33300000', 'status': 'FILLED', 'timeInForce': 'FOK', 'type': 'LIMIT', 'side': 'SELL', 'fills': [{'price': '2903.33000000', 'qty': '0.10000000', 'commission': '0.29033300', 'commissionAsset': 'EUR', 'tradeId': 45712569}]}
order {'symbol': 'ETHEUR', 'orderId': 965165814, 'orderListId': -1, 'clientOrderId': 'JrQ

{'symbol': 'ETHEUR',
 'orderId': 965165814,
 'orderListId': -1,
 'clientOrderId': 'JrQOEd4XdpNlznfEAEyDbQ',
 'price': '2901.60000000',
 'origQty': '0.10000000',
 'executedQty': '0.10000000',
 'cummulativeQuoteQty': '290.33300000',
 'status': 'FILLED',
 'timeInForce': 'FOK',
 'type': 'LIMIT',
 'side': 'SELL',
 'stopPrice': '0.00000000',
 'icebergQty': '0.00000000',
 'time': 1631999318981,
 'updateTime': 1631999318981,
 'isWorking': True,
 'origQuoteOrderQty': '0.00000000'}

In [2]:
from strategies import EmaStrategy
from connectors import BinanceConnector

ema_strategy = EmaStrategy()
df = ema_strategy._collect_data()

In [3]:
df.tail()

,low,high,open,close,engulfing,ema_3,ema_6,ema_9,sma_21,sma_50,sma_200
timestamp_utc,,,,,,,,,,,
2021-09-29 20:15:00+00:00,2407.27,2416.28,2414.44,2412.25,0,2418.187370,2423.296948,2426.174527,NaN,NaN,NaN
2021-09-29 20:20:00+00:00,2411.01,2426.44,2412.92,2424.69,0,2421.438685,2423.694963,2425.877621,NaN,NaN,NaN
2021-09-29 20:25:00+00:00,2414.01,2422.48,2422.48,2417.61,0,2419.524343,2421.956402,2424.224097,NaN,NaN,NaN
2021-09-29 20:30:00+00:00,2411.31,2425.71,2416.13,2422.70,100,2421.112171,2422.168859,2423.919278,NaN,NaN,NaN
2021-09-29 20:35:00+00:00,2421.03,2429.08,2421.03,2426.71,0,2423.911086,2423.466328,2424.477422,2430.40381,NaN,NaN


In [13]:
df[-1:]

NameError: name 'df' is not defined

In [13]:
ema_strategy._get_current_status()

TypeError: _get_current_status() missing 1 required positional argument: 'last_transaction'

In [1]:
from connectors import DummyConnector
simulator = DummyConnector()
simulator.name

TypeError: object of type 'NoneType' has no len()

In [2]:
from connectors import DummyConnector
dummy_connector = DummyConnector()


TypeError: non-default argument 'account_balance' follows default argument

In [1]:
from connectors import BinanceConnector
binance_connector = BinanceConnector()

client <binance.client.Client object at 0x7fa77c522790>
Oops!  Something went wrong with accessing the account


BinanceAPIException: APIError(code=-1021): Timestamp for this request was 1000ms ahead of the server's time.

In [15]:
import numpy as np
def trendline(data, order=1):
    index = range(len(data))
    coeffs = np.polyfit(index, list(data), order)
    slope = coeffs[-2]
    return float(slope)


List=[3466.6825,3466.5982,3466.7700,3466.8563]
resultent=trendline(List)
print(resultent)

0.06931999999968395


In [14]:
from strategies import EmaStrategy
from connectors import BinanceConnector

ema_strategy = EmaStrategy()
df = ema_strategy._collect_data()

ConnectionError: HTTPConnectionPool(host='localhost', port=8086): Max retries exceeded with url: /query?q=SELECT+time%2C+exchange%2C+pair%2C+price+FROM+binance_live_trades+WHERE+time+%3E%3D+now%28%29-+1d+and+time+%3C%3D+now%28%29+order+by+time+desc&db=pi_influxdb (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fa741698350>: Failed to establish a new connection: [Errno 111] Connection refused'))